In [1]:
import glob
import os
import re

In [3]:
# list of dirs with job
target_dirs = [r'E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs', ]
out_dir = r'out\path\dir'

files = []
for target_dir in target_dirs:
    files += glob.glob(target_dir + '\\*.txt')

# slavery weight fix
weights_list = [
    '''
		modifier = {
			factor = 0.25
			is_enslaved = yes
			can_take_servant_job = no
			NOR = { 
				has_trait = trait_nuumismatic_administration 
				has_trait = trait_thrifty
			}
		}''',
    '''
		modifier = {
			factor = 2
			OR = {
				is_non_sapient_robot = yes
				is_shackled_robot = yes
			}
			can_take_servant_job = no
			owner = { has_technology = tech_droid_workers }
		}''',
    '''
		modifier = {
			factor = 200
			OR = {
				is_non_sapient_robot = yes
				is_shackled_robot = yes
			}
			can_take_servant_job = no
			owner = { NOT = { has_technology = tech_droid_workers } }
		}''',
    '''
		modifier = {
			factor = 10
			is_enslaved = yes
			can_take_servant_job = no
		}''',
    '''
		modifier = {
			factor = 2
			OR = {
				is_non_sapient_robot = yes
				is_shackled_robot = yes
			}
			can_take_servant_job = no
		}''',
    '''
		modifier = {
			factor = 0.25
			is_enslaved = yes
			can_take_servant_job = no
			NOR = {
				has_trait = trait_nuumismatic_administration
				has_trait = trait_thrifty
			}
		}''',
    '''
		modifier = {
			factor = 8
			is_enslaved = yes
			can_take_servant_job = no
			NOR = {
				has_trait = trait_syncretic_proles
				has_trait = trait_nuumismatic_administration
				has_trait = trait_robot_superconductive
				has_trait = trait_ingenious
			}
		}''',
]

slave_robot_weight = '''		modifier = {
			factor = 10
			OR = {
				is_enslaved = yes
				is_non_sapient_robot = yes
				is_shackled_robot = yes
			}
			can_take_servant_job = no
		}'''

def jobs(files):
    for file in files:
        print(file)
        out_file = os.path.join(out_dir, os.path.basename(file))
        with open(file, 'r') as f:
            text = f.read()
        #text = re.sub('#+.*\n', '\n', text)
        text = re.sub('\t*\n', '\n', text)
        text = re.sub(' *\n', '\n', text)
        things = re.findall(r'\n\w*? = {.*?\n}', text, re.DOTALL)
        with open(out_file, 'w') as f:
            for thing in things:
                name = ''
                name = re.match(r'\n\w* = {', thing).group(0).replace(' = {', '').replace('\n', '')
                if 'is_capped_by_modifier = no' not in thing:
                    thing = thing.replace('possible = {', 'possible = {{\n\t\tor = {{\n\t\t\thas_job = {}\n\t\t\tdpcb_trigger_job = yes\n\t\t}}'.format(name), 1)
                    if any(i in thing for i in weights_list):
                        for i in weights_list:
                            thing = thing.replace(i, '')

                        weight = re.search('\n\tweight = {\n.*', thing).group(0)
                        thing = thing.replace(weight, weight+'\n'+slave_robot_weight)
                f.write(thing)
                f.write('\n')

jobs(files)

E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\000_pretriggers.txt
E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\00_other_jobs.txt
E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\01_ruler_jobs.txt
E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\02_specialist_jobs.txt
E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\03_worker_jobs.txt
E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\04_gestalt_jobs.txt
E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\05_primitive_jobs.txt
E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\06_event_jobs.txt
E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs\07_fallen_empire_jobs.txt


In [ ]:
# # 
# target_dir = r'E:\Program Files (x86)\Steam\steamapps\common\Stellaris\common\pop_jobs'

# def capped_jobs(files):
#     capped_jobs_list = []
#     for file in files:
#         with open(file, 'r') as f:
#             text = f.read()
#         #text = re.sub('#+.*\n', '\n', text)
#         text = re.sub('\t*\n', '\n', text)
#         text = re.sub(' *\n', '\n', text)
#         things = re.findall(r'\n\w*? = {.*?\n}', text, re.DOTALL)
#         for thing in things:
#             name = ''
#             name = re.match(r'\n\w* = {', thing).group(0).replace(' = {', '').replace('\n', '')
#             #print(name)
#             if 'is_capped_by_modifier = no' in thing and 'purge' not in name and 'has_slavery_type' not in thing:
#                 capped_jobs_list.append(name)
#     return capped_jobs_list

# files = glob.glob(target_dir + '\\*.txt')[:5]
# capped_jobs_list = capped_jobs(files)
# capped_jobs_list